In [16]:
import pandas as pd

cleaned_orders_reviews_df = pd.read_csv("cleaned_orders_reviews")

cleaned_orders_reviews_df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


### Since we are only focussing on review comment message then we are dropping the rest and store it into a new dataframe

In [ ]:
#drop the null values that are in review_comment_message and reset index and drop the old index column
text_analysis_df = cleaned_orders_reviews_df.dropna(subset=['translated_review_comment']).reset_index(drop=True)

#text_analysis_df.head()

text_analysis_df = text_analysis_df.loc[:,["review_id","order_id", "review_score", "translated_comment_title" ,"translated_review_comment", "review_creation_date"]]

text_analysis_df.head()



,review_id,order_id,review_score,translated_comment_title,translated_review_comment,review_creation_date
0,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,I received it well before the stipulated deadl...,2017-04-21
1,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Congratulations lannister stores I loved shopp...,2018-03-01
2,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,I recommend,efficient device. On the website the brand of ...,2018-05-22
3,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,NaN,"But a little slow...for the price, it's good.",2018-02-16
4,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,I highly recommend,"Reliable seller, ok product and delivery on time.",2018-05-23


or

## Keyword Extraction

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from collections import Counter

# Download NLTK resources (run once)
nltk.download('punkt')
nltk.download('stopwords')

# Function to extract keywords from the text
def extract_keywords(text, num_keywords=5):
    # Tokenization
    tokens = word_tokenize(text.lower())
    
    # Removing stopwords removing words that is non-aphabetical as well as words that exist in stop words like i.e the
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    
    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Frequency count
    word_freq = nltk.FreqDist(tokens)

    # Create dictionary
    word_dict = {word: idx for idx, (word, _) in enumerate(word_freq.items())}

    return word_dict

    # # keyword_counter = Counter(filtered_tokens)
    # return keyword_counter.most_common(num_keywords)

# Apply keyword extraction to the 'Comment' column
text_analysis_df['Keywords'] = text_analysis_df['translated_review_comment'].apply(extract_keywords)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\T470\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\T470\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
text_analysis_df.head(10)

,review_id,order_id,review_score,translated_comment_title,translated_review_comment,review_creation_date,Sentiment,Keywords
0,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,I received it well before the stipulated deadl...,2017-04-21,Neutral,"{'i': 0, 'receiv': 1, 'it': 2, 'well': 3, 'bef..."
1,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Congratulations lannister stores I loved shopp...,2018-03-01,Positive,"{'congratul': 0, 'lannist': 1, 'store': 2, 'i'..."
2,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,I recommend,efficient device. On the website the brand of ...,2018-05-22,Neutral,"{'effici': 0, 'devic': 1, '.': 2, 'on': 3, 'th..."
3,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,NaN,"But a little slow...for the price, it's good.",2018-02-16,Positive,"{'but': 0, 'a': 1, 'littl': 2, 'slow': 3, '......"
4,3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,I highly recommend,"Reliable seller, ok product and delivery on time.",2018-05-23,Positive,"{'reliabl': 0, 'seller': 1, ',': 2, 'ok': 3, '..."
5,9314d6f9799f5bfba510cc7bcd468c01,0dacf04c5ad59fd5a0cc1faa07c34e39,2,NaN,"I WOULD LIKE TO KNOW WHAT HAPPENED, I ALWAYS R...",2018-01-18,Negative,"{'i': 0, 'would': 1, 'like': 2, 'to': 3, 'know..."
6,373cbeecea8286a2b66c97b1b157ec46,583174fbe37d3d5f0d6661be3aad1786,1,My product didn't arrive,Terrible,2018-08-15,Negative,{'terribl': 0}
7,d21bbc789670eab777d27372ab9094cc,4fc44d78867142c627497b60a7e0228a,5,Excellent,Store note 10,2018-07-10,Neutral,"{'store': 0, 'note': 1, '10': 2}"
8,0e0190b9db53b689b285d3f3916f8441,79832b7cb59ac6f887088ffd686e1d5e,5,NaN,thank you for the attention you gave me,2017-12-01,Neutral,"{'thank': 0, 'you': 1, 'for': 2, 'the': 3, 'at..."
9,fe3db7c069d694bab50cc43463f91608,2ca73e2ff9e3a186ad1e1ffb9b1d9c10,5,NaN,The purchase was made easily. Delivery was mad...,2018-03-23,Positive,"{'the': 0, 'purchas': 1, 'wa': 2, 'made': 3, '..."


# wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

def visualize_word_cloud(text):
    # Generate word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    
    # Display the generated image:
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.show()

# Visualize word cloud
visualize_word_cloud(text_analysis_df['Keywords'])

TypeError: expected string or bytes-like object, got 'Series'

## Topic Modelling

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Function to perform topic modeling using LDA
def perform_topic_modeling(texts, num_topics=5):
    vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
    X = vectorizer.fit_transform(texts)
    lda = LatentDirichletAllocation(n_components=num_topics, random_state=42)
    lda.fit(X)
    return lda

# Apply topic modeling to the 'Comment' column
lda_model = perform_topic_modeling(text_analysis_df['translated_review_comment'])
